In [ ]:
## load MD_work and rename it to "MD_work_CPU"

In [ ]:
### KVD(ref)-EBNA1- MMPBSA

In [1]:
# Step 1: Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

# Set install path inside Drive
import os
gromacs_path = "/content/drive/MyDrive/gromacs-2021.4-CPU"

# Check if already installed
if not os.path.exists(gromacs_path):
    print("🚀 Installing CPU-only GROMACS (no GPU)... (~15–20 min)")
    !apt-get update -qq
    !apt-get install -y build-essential cmake git wget unzip libfftw3-dev libgsl-dev

    # Download and extract GROMACS
    !wget -q http://ftp.gromacs.org/pub/gromacs/gromacs-2021.4.tar.gz -O gromacs-2021.4.tar.gz
    !tar -xf gromacs-2021.4.tar.gz
    %cd gromacs-2021.4

    !mkdir build
    %cd build

    # ✅ CPU-ONLY BUILD (NO CUDA, NO GPU FLAGS)
    !cmake .. \
        -DGMX_BUILD_OWN_FFTW=ON \
        -DGMX_GPU=OFF \
        -DREGRESSIONTEST_DOWNLOAD=ON \
        -DCMAKE_INSTALL_PREFIX={gromacs_path}

    !make -j4
    !make install
    print("✅ CPU-only GROMACS installed!")

    %cd /content
    !rm -rf gromacs-2021.4 gromacs-2021.4.tar.gz

else:
    print("✅ CPU-only GROMACS already installed — skipping build.")

# Add GROMACS to PATH
os.environ["PATH"] = f"{gromacs_path}/bin:" + os.environ["PATH"]

# MD work directory
work_dir = "/content/drive/MyDrive/MD_work_CPU"
os.makedirs(work_dir, exist_ok=True)
%cd $work_dir

✅ CPU-only GROMACS already installed — skipping build.
/content/drive/MyDrive/MD_work_CPU


In [3]:

!chmod -R +x {gromacs_path}/bin

print("✅ Working directory set to:", work_dir)
!gmx --version

✅ Working directory set to: /content/drive/MyDrive/MD_work_CPU
                         :-) GROMACS - gmx, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pa

In [4]:
# Install Conda in Colab
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...


In [1]:
!conda install -c conda-forge gmx_mmpbsa

Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - gmx_mmpbsa


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ambertools-23.6            |cuda_None_nompi_py311h4a53416_105        90.2 MB  conda-forge
    arpack-3.9.1               |nompi_hf03ea27_102         127 KB  conda-forge
    blosc-1.21.6               |       he440d0b_1          47 KB  conda-forge
    brotli-1.1.0               |       hb9d3cd8_2          19 KB  conda-forge
    brotli-bin-1.1.0           |       hb9d3cd8_2          18 KB  conda-forge
    ca-certificates-2025.11.12 |       hbd8a1cb_0         149 KB  conda-forge
    cached-property-1.5.2      |       hd8ed1ab_1           4 KB  conda-forge
    cached_property-1.5.2      |     pyha770c72_1          11 KB  conda-forge
    certifi-2025.11.12      

In [2]:
import os
# This points to your specific CPU-only build path
gromacs_bin = "/content/drive/MyDrive/gromacs-2021.4-CPU/bin"

# Add it to the system PATH
os.environ["PATH"] = gromacs_bin + ":" + os.environ["PATH"]
os.environ["GMXBIN"] = gromacs_bin

In [3]:
# Verify both tools are ready
print("Checking GROMACS...")
!gmx --version | grep "GROMACS version"
print("\nChecking gmx_MMPBSA...")
!gmx_MMPBSA --version

Checking GROMACS...
GROMACS version:    2021.4

Checking gmx_MMPBSA...
[INFO   ] Starting gmx_MMPBSA v1.5.0.3
[INFO   ] Command-line
  gmx_MMPBSA --version

gmx_MMPBSA v1.5.0.3 based on MMPBSA version 16.0 and AmberTools 20


In [4]:
import os
work_dir = "/content/drive/MyDrive/MD_work_CPU"
os.chdir(work_dir)

In [5]:
# 1. The Python Script (To generate mmpbsa.in)
# Run this block in your notebook. It will create the instruction file.
mmpbsa_input = """
&general
  sys_name="Prot_Lig_Complex",
  startframe=3100,
  endframe=3600,
  interval=1,
/
&gb
  igb=2,
  saltcon=0.15,
/
"""
with open("mmpbsa.in", "w") as f:
    f.write(mmpbsa_input)

print("✅ mmpbsa.in has been fixed (forcefield variable removed)!")

✅ mmpbsa.in has been fixed (forcefield variable removed)!


In [6]:

# 2. The Execution Command
# After running the Python script above, run this command in your terminal (or a code cell with ! if in Colab).

# Note: You need to specify your index groups. Based on your previous output, Group 1 is Protein and Group 13 is LIG.
                                                                          # Takes 6-7 min
!gmx_MMPBSA -O -i mmpbsa.in -cs md_0_10.tpr -ct md_1_final.xtc -ci index.ndx -cg 1 13 -cp topol.top

[INFO   ] Starting gmx_MMPBSA v1.5.0.3
[INFO   ] Command-line
  gmx_MMPBSA -O -i mmpbsa.in -cs md_0_10.tpr -ct md_1_final.xtc -ci index.ndx -cg 1 13 -cp topol.top

[INFO   ] Checking external programs...
[INFO   ] cpptraj found! Using /usr/local/bin/cpptraj
[INFO   ] tleap found! Using /usr/local/bin/tleap
[INFO   ] parmchk2 found! Using /usr/local/bin/parmchk2
[INFO   ] sander found! Using /usr/local/bin/sander
[INFO   ] Using GROMACS version > 5.x.x!
[INFO   ] gmx found! Using /content/drive/MyDrive/gromacs-2021.4-CPU/bin/gmx
[INFO   ] Checking external programs...Done.

[INFO   ] Building AMBER topologies from GROMACS files...
[INFO   ] Get PDB files from GROMACS structures files...
[INFO   ] Making gmx_MMPBSA index for complex...
[INFO   ] Normal Complex: Saving group 1_13 in _GMXMMPBSA_COM_index.ndx file as _GMXMMPBSA_COM.pdb
[INFO   ] No receptor structure file was defined. Using ST approach...
[INFO   ] Using receptor structure from complex to generate AMBER topology
[INFO   ] N

In [10]:
import plotly.graph_objects as go

# Data extracted from your specific Delta (Complex - Receptor - Ligand) results
categories = ['vdW (ΔVDWAALS)', 'Electrost. (ΔEEL)', 'Polar Solv (ΔEGB)', 'Non-polar (ΔESURF)']
values = [-20.77, -2.41, 13.51, -3.35]
total_delta_g = -13.02

# Create the figure
fig = go.Figure()

# Add the individual energy components
fig.add_trace(go.Bar(
    x=categories,
    y=values,
    marker_color=['#1f77b4', '#d62728', '#2ca02c', '#9467bd'],
    text=[f"{v} kcal/mol" for v in values],
    textposition='auto',
    name='Energy Components'
))

# Add a dashed line for the Net Binding Energy (Total Delta G)
fig.add_shape(
    type="line", line=dict(color="Black", width=3, dash="dash"),
    x0=-0.5, x1=3.5, y0=total_delta_g, y1=total_delta_g
)

# Add annotation for the Total Delta G
fig.add_annotation(
    x=1.5, y=total_delta_g - 5,
    text=f"<b>Net ΔG bind: {total_delta_g} kcal/mol</b>",
    showarrow=True,
    arrowhead=1,
    font=dict(size=14, color="black"),
    bgcolor="white",
    bordercolor="black"
)

# Formatting the layout
fig.update_layout(
    title='<b>Binding Free Energy Components (MM/GBSA)</b><br><sup>Results for Prot_Lig_Complex</sup>',
    yaxis_title='Δ Energy (kcal/mol)',
    template='plotly_white',
    showlegend=False,
    height=600,
    width=900,
    yaxis=dict(zeroline=True, zerolinewidth=2, zerolinecolor='black')
)

# Add horizontal line at zero for clarity
fig.add_hline(y=0, line_width=2, line_color="black")

fig.show()

In [11]:
## input file created to find hotspot binding residues
mmpbsa_input = """
&general
  sys_name="Prot_Lig_Decomp",
  startframe=3100,
  endframe=3600,
  interval=10,
/
&gb
  igb=2,
  saltcon=0.15,
/
&decomp
  idecomp=1,
  print_res="within 5",
/
"""
with open("mmpbsa_decomp.in", "w") as f:
    f.write(mmpbsa_input)

print("✅ Decomposition input file created!")

✅ Decomposition input file created!


In [13]:
!gmx_MMPBSA -O -i mmpbsa_decomp.in -cs md_0_10.tpr -ct md_1_final.xtc -ci index.ndx -cg 1 13 -cp topol.top

[INFO   ] Starting gmx_MMPBSA v1.5.0.3
[INFO   ] Command-line
  gmx_MMPBSA -O -i mmpbsa_decomp.in -cs md_0_10.tpr -ct md_1_final.xtc -ci index.ndx -cg 1 13 -cp topol.top

[INFO   ] Checking external programs...
[INFO   ] cpptraj found! Using /usr/local/bin/cpptraj
[INFO   ] tleap found! Using /usr/local/bin/tleap
[INFO   ] parmchk2 found! Using /usr/local/bin/parmchk2
[INFO   ] sander found! Using /usr/local/bin/sander
[INFO   ] Using GROMACS version > 5.x.x!
[INFO   ] gmx found! Using /content/drive/MyDrive/gromacs-2021.4-CPU/bin/gmx
[INFO   ] Checking external programs...Done.

[INFO   ] Building AMBER topologies from GROMACS files...
[INFO   ] Get PDB files from GROMACS structures files...
[INFO   ] Making gmx_MMPBSA index for complex...
[INFO   ] Normal Complex: Saving group 1_13 in _GMXMMPBSA_COM_index.ndx file as _GMXMMPBSA_COM.pdb
[INFO   ] No receptor structure file was defined. Using ST approach...
[INFO   ] Using receptor structure from complex to generate AMBER topology
[INF

In [14]:
import os

# Search for the decomposition file
files = [f for f in os.listdir('.') if 'DECOMP' in f.upper() and f.endswith('.dat')]

if not files:
    print("❌ No decomposition .dat file found. Please run 'ls' to see your file list.")
else:
    target_file = files[0]
    print(f"✅ Found file: {target_file}\n")

    with open(target_file, 'r') as f:
        lines = f.readlines()

    # Find the summary table in the .dat file
    printing = False
    for line in lines:
        if "Total Energy Decomposition:" in line or "Residue" in line:
            printing = True
        if printing:
            print(line.strip())
        if "Total" in line and printing and len(line.strip()) == 0:
            break

✅ Found file: FINAL_DECOMP_MMPBSA.dat

Total Energy Decomposition:
Residue,Internal,,,van der Waals,,,Electrostatic,,,Polar Solvation,,,Non-Polar Solv.,,,TOTAL,,
,Avg.,Std. Dev.,Std. Err. of Mean,Avg.,Std. Dev.,Std. Err. of Mean,Avg.,Std. Dev.,Std. Err. of Mean,Avg.,Std. Dev.,Std. Err. of Mean,Avg.,Std. Dev.,Std. Err. of Mean,Avg.,Std. Dev.,Std. Err. of Mean
R:B:ASN:480,0.0,6.34875335185146,0.8890032877003671,-0.06701960784313726,0.9529737039554679,0.1334430098251246,-0.09792156862745084,10.933896159909265,1.5310517033551279,0.11190196078431373,6.589473127003651,0.9227107983980984,-0.005544847058823528,0.07307284054507117,0.010232244329861761,-0.05858406274509789,14.289555940276202,2.0009380592772907
R:B:ILE:481,0.0,3.3777455994895766,0.4729789891878504,-1.0731960784313725,1.447895337150904,0.20274590043694313,-0.0599803921568627,1.776826466972276,0.24880547144751303,0.3488039215686275,0.9541027653715598,0.13360111004658443,-0.1599696,0.07726065972912691,0.01081865630975613,-0.94434214

In [18]:
import plotly.graph_objects as go
import pandas as pd

# 1. Dataset extracted from your FINAL_DECOMP_MMPBSA.dat
data = {
    'Residue': ['ASN-519', 'THR-515', 'ILE-481', 'SER-516', 'THR-590',
                'LEU-520', 'LYS-586', 'LEU-582', 'VAL-583', 'ILE-593'],
    'Total': [-1.25, -0.95, -0.94, -0.90, -0.75, -0.55, -0.41, -0.39, -0.29, -0.17],
    'VDW': [-1.39, -0.99, -1.07, -1.15, -0.96, -0.75, -0.65, -0.46, -0.35, -0.24],
    'EEL': [-0.33, -0.35, -0.06, -0.34, -0.48, 0.16, 0.76, 0.05, 0.02, 0.02]
}

df = pd.DataFrame(data).sort_values(by='Total')

# 2. Create the Interactive Plot
fig = go.Figure()

fig.add_trace(go.Bar(
    x=df['Residue'],
    y=df['Total'],
    marker_color='royalblue',
    text=df['Total'],
    textposition='outside',
    name='Total Delta G',
    customdata=df[['VDW', 'EEL']],
    hovertemplate="<b>%{x}</b><br>Total: %{y} kcal/mol<br>VDW: %{customdata[0]}<br>EEL: %{customdata[1]}<extra></extra>"
))

# 3. Format the layout
fig.update_layout(
    title='<b>Per-Residue Energy Decomposition (Top 10)</b>',
    xaxis_title='Residue (Chain B)',
    yaxis_title='Binding Energy (kcal/mol)',
    template='plotly_white',
    height=600,
    yaxis=dict(range=[min(df['Total']) - 0.5, 1.0]) # Adjust range to see positive EEL values if needed
)

fig.add_hline(y=0, line_width=2, line_color="black")
fig.show()

In [15]:
import pandas as pd
import plotly.graph_objects as go
import os

file_path = 'FINAL_DECOMP_MMPBSA.dat'

if not os.path.exists(file_path):
    print(f"Error: {file_path} not found!")
else:
    with open(file_path, 'r') as f:
        lines = f.readlines()

    start_index = 0
    for i, line in enumerate(lines):
        if line.startswith("Residue,Internal"):
            start_index = i
            break

    # Read the data
    df = pd.read_csv(file_path, skiprows=start_index + 1, header=None)

    # Process the data and FORCE numeric conversion
    plot_df = pd.DataFrame({
        'Residue': df[0].str.replace('R:B:', '').str.replace('L:C:', '').str.replace(':', '-'),
        'Total': pd.to_numeric(df.iloc[:, -3], errors='coerce'),
        'StdDev': pd.to_numeric(df.iloc[:, -2], errors='coerce'),
        'VDW': pd.to_numeric(df[4], errors='coerce'),
        'EEL': pd.to_numeric(df[7], errors='coerce')
    })

    # Drop any rows that failed to convert (like footer text)
    plot_df = plot_df.dropna(subset=['Total'])

    # Exclude the Ligand itself (LIG-608) so we only see protein contributions
    # Comment out the next line if you want to include the ligand bar
    plot_df = plot_df[~plot_df['Residue'].str.contains('LIG')]

    # Sort so the strongest binders (most negative) are on the left
    plot_df = plot_df.sort_values(by='Total')

    # Create Interactive Plotly Figure
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=plot_df['Residue'],
        y=plot_df['Total'],
        error_y=dict(type='data', array=plot_df['StdDev'], visible=True),
        marker=dict(
            color=plot_df['Total'], # Now it's a numeric array!
            colorscale='Blues',
            reversescale=True,
            line=dict(width=1, color='black')
        ),
        customdata=plot_df[['VDW', 'EEL']],
        hovertemplate="<b>%{x}</b><br>" +
                      "Total ΔG: %{y:.2f} kcal/mol<br>" +
                      "VDW: %{customdata[0]:.2f}<br>" +
                      "EEL: %{customdata[1]:.2f}<br>" +
                      "<extra></extra>"
    ))

    fig.update_layout(
        title='Per-Residue Energy Decomposition: Binding Hotspots',
        xaxis_title='Residue Name',
        yaxis_title='Energy Contribution (kcal/mol)',
        template='plotly_white',
        xaxis_tickangle=-45,
        height=600
    )

    fig.add_hline(y=0, line_width=2, line_color="black")

    fig.show()

In [16]:
!grep -A 20 "Delta (Complex - Receptor - Ligand)" FINAL_RESULTS_MMPBSA.dat

Delta (Complex - Receptor - Ligand):
Energy Component       Average     SD(Prop.)         SD   SEM(Prop.)        SEM
-------------------------------------------------------------------------------
ΔBOND                    -0.00          2.36       0.00         0.33       0.00
ΔANGLE                   -0.00          2.19       0.00         0.31       0.00
ΔDIHED                   -0.00          1.39       0.00         0.19       0.00
ΔUB                       0.00          0.00       0.00         0.00       0.00
ΔIMP                     -0.00          0.31       0.00         0.04       0.00
ΔCMAP                     0.00          0.00       0.00         0.00       0.00
ΔVDWAALS                -20.91          1.55       7.09         0.22       0.99
ΔEEL                     -3.01          0.49       5.13         0.07       0.72
Δ1-4 VDW                  0.00          1.42       0.00         0.20       0.00
Δ1-4 EEL                  0.00          2.80       0.00         0.39       0.00
ΔEG